In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyabf

Making this notebook to show James what I have been doing so far in a consolidated format

Looking at the waveforms (from Plotting folder)

Annotations sheet --(app script)> unformatted csv files 

unformatted csv files --(excell processor)> formatted csv files 

`annotations_loader` : Make `sheets` dict with 38 dataframes of all processed .csvs

In [ ]:
cell_types_df = pd.read_csv("/Users/Haley/Desktop/Neuroscience Lab/Repository/murray-neuroscience-lab/Excel processor/List of cells.csv")
cell_types_df = cell_types_df.dropna(how='all')
cell_types_df = cell_types_df.iloc[:,:4]
cell_types_df.reset_index(drop=True,inplace=True)
sheets_types = cell_types_df["Cell"].unique().tolist()

In [ ]:
sheet_names_df = pd.read_csv('/Users/Haley/Desktop/Neuroscience Lab/Repository/murray-neuroscience-lab/Processed CSVs/sheet_names.csv', header=None)
sheet_names = sheet_names_df.iloc[:,0].values
sheets = {}

for sheet in sheet_names:
    file_path = "/Users/Haley/Desktop/Neuroscience Lab/Repository/murray-neuroscience-lab/Processed CSVs/"+sheet+".csv"
    df = pd.read_csv(file_path)
    df[["Trace name","Tags","Type"]] = df[["Trace name","Tags","Type"]].astype("string")
    if sheet in sheets_types:
        types = cell_types_df[cell_types_df["Cell"]==sheet]
        df["Cell Type"] = types.iloc[0][1]
        df["Input Resistance"] = types.iloc[0][2]
        df["Motoneuron"] = types.iloc[0][3]
    sheets[sheet] = df

sheets

`binwaves_` : make abfs dict to store .abf files for each of the 38 sheets

make `abfs` dict: contains the abf file of each trace in the annotation df sheet

In [ ]:
traces = df["Trace name"].unique().tolist()
abfs = {}

for trace in traces:
    abfs[trace] = []
    file_path2 = "/Users/Haley/Downloads/.abf files annotated/" + trace +".abf"
    abf = pyabf.ABF(file_path2)
    abfs[trace]= abf

In [ ]:
def make_waveforms(abf,df):
    '''
    Function that takes an abf file and a df of the annotations
    Returns a dictionary with waveforms labeled by their frequency
    '''
    abf.setSweep(sweepNumber=0, channel=0)
    x = abf.sweepX
    y = abf.sweepY
    abf_df = pd.DataFrame({
        'Time': x,
        'Current': y
    })
    # df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
    # df["Seconds"] = df['On time']*0.001
    waveforms = {}
    for i in range(len(df) - 1):
        t_0 = df.iloc[i,6] 
        t_f = df.iloc[i+1,6]
        phase_0 = int(t_0 * 100000)
        phase_1 = int(t_f * 100000)
        abf_waveform = abf_df.loc[phase_0:phase_1]
        phase = (abf_waveform["Time"] - t_0) / (t_f - t_0)
        abf_waveform["Phase"] = phase
        
        # NORMALIZE CURRENT ALSO, test this:
        y_max = max(abf_waveform["Current"])
        y_min = min(abf_waveform["Current"])
        normalized = (abf_waveform["Current"] - y_min) / (y_max - y_min)
        abf_waveform["Normalized Current"] = normalized
        #=====

        # Dict keys
        freq = 1 / (t_f-t_0) # CHECK = df.loc[i + 1, "Freq"]
        cell_type = df.iloc[0,5]
        signal_type = df.iloc[0,7]
        fast_slow = df.iloc[0,8:9] # 8:resistance 9:prim/sec low/high rin need to see what is fast/slow
        key = (freq,cell_type,signal_type,fast_slow)
        waveforms[key] = abf_waveform

    return waveforms

need abfs[trace] and sheets[sheet[sheet["Trace name"]==Trace]]

use loops in annotations loader to make mega dict

TEST: add nested dicts with abf files into `sheets` 

In [ ]:
for sheet in sheets.keys():
    df = sheets[sheet]
    traces = df["Trace name"].unique().tolist()
    abfs = {}
    for trace in traces:
        abfs[trace] = []
        file_path2 = "/Users/Haley/Downloads/.abf files annotated/" + trace +".abf"
        abf = pyabf.ABF(file_path2)
        abfs[trace]= abf
    sheets[sheet] = {
        "annotations": sheets[sheet],
        "abfs": abfs
    }

TEST: loop to make a mega dict `all_waveforms`

In [ ]:
all_waveforms = {}
for sheet in sheets.keys():
    traces_df_mkwf = sheets[sheet]["annotations"]
    abfs_mkwv = sheets[sheet]["abfs"]
    traces = traces_df_mkwf["Trace name"].unique().tolist()
    for trace in traces:
        abf_mkwv = abfs_mkwv[trace]
        df_mkwv = traces_df_mkwf[traces_df_mkwf["Trace name"]==trace]
        waveforms = make_waveforms(abf_mkwv,df_mkwv)

        # NEED TO FIGURE OUT HOW TO JOIN WAVEFORMS W ALL WAVEFORMS DICTIONARIES HERE

In [ ]:
def bin_wave(onewave):
    # Make edges for the bin
    bins = np.linspace(0, 1, 51, endpoint = True)

    # Cut the phase into bins
    onewave['Phase Bin'] = pd.cut(onewave['Phase'], bins=bins, include_lowest=True)
    # print(onewave)

    # Group by the bins and calculate the average current
    # binned_avg = onewave.groupby(by='Phase Bin','Current')
    binned_avg = onewave.groupby('Phase Bin')['Current'].mean().reset_index()
    # print(binned_avg)

    # Get phase column for plotting (middle of bin)
    phase = np.arange(0.01, 1.00001, 0.02)
    # len(phase)
    binned_avg['Phase'] = phase
    return binned_avg

bin `all_waveforms`

In [ ]:
all_waveforms = make_waveforms(abf, annotation_df)
onewave = all_waveforms[np.float64(35.906642728904366)]
binned_waveforms = {}
for wave in all_waveforms.keys():
    binned = bin_wave(all_waveforms[wave])
    binned_waveforms[wave] = binned

NEXT: access keys to average across the key types and plot
- current type: excitatory, inhibitory, cell-attached
- frequency bins:
- rin: low/mid/high or fast/slow
- cell type: CaP,vRoP,MiP,dRoP,dVS,dSMN,vSMN,iSMN

function(current,freq,rin,type) -> plot waveform